In [1]:
#Imports
library(ROCR)
library(PRROC)
library(ggplot2)
library(gridGraphics)
library(grid)
library(ggpubr)
library(gridExtra)
library(grid)

Loading required package: gplots

Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess

Loading required package: grid
Loading required package: magrittr


In [38]:
#Constants
test_type <- "rna_binding"
#All data paths
pred_path <- paste0("/home/anat/Research/ExAC/14.Final_Model/2nd_level_pred/",test_type,"/")
drnapred_path <- paste0("/home/anat/Research/ExAC/14.Final_Model/Competitors_pred/",test_type,"/")

ligands_list <- c("rna")
folds_num <- 1
colors_list <- c("grey", "black")
model_names <- c("XGB", "DRNApred")

In [39]:
#===Read the ligands datasets==#
ligand_data <- list()
for (i in 1:length(ligands_list)) {
    ligand <- ligands_list[[i]]
    ligand_filename <- paste0(ligand, "_XGB.csv")
    ligand_data[[i]] <- read.csv(paste0(pred_path, ligand_filename), header = TRUE, sep = '\t', row.names = 1, stringsAsFactors=FALSE)
}
# Add DRNApred data
for (i in 1:length(ligands_list)) {     
    ligand <- ligands_list[[i]]
    ligand_filename <- paste0(ligand, "_DRNApred.csv")
    ligand_comp_data <- read.csv(paste0(drnapred_path, ligand_filename), header = TRUE, sep = '\t', row.names = 1, stringsAsFactors=FALSE)
    ligand_data[[i]] <- rbind(ligand_data[[i]], ligand_comp_data)
}
    

#==Read the domain ligands datasets==#
domain_ligand_data <- list()
for (i in 1:length(ligands_list)) {
    ligand <- ligands_list[[i]]
    ligand_filename <- paste0(ligand, "_XGB_d.csv")
    domain_ligand_data[[i]] <- read.csv(paste0(pred_path, ligand_filename), header = TRUE, sep = '\t', row.names = 1, stringsAsFactors=FALSE)
}
# Add DRNApred stacked data
for (i in 1:length(ligands_list)) {
    ligand <- ligands_list[[i]]
    ligand_filename <- paste0(ligand, "_DRNApred_d.csv")
    domain_ligand_comp_data <- read.csv(paste0(drnapred_path, ligand_filename), header = TRUE, sep = '\t', row.names = 1, stringsAsFactors=FALSE)
    domain_ligand_data[[i]] <- rbind(domain_ligand_data[[i]], domain_ligand_comp_data)
}

#==Read the AUPRC tables==#
auprc_data <- list()
for (i in 1:length(ligands_list)) {
    ligand <- ligands_list[[i]]
    auprc_filename <- paste0(ligand, "_XGB_auprc.csv")
    auprc_data[[i]] <- read.csv(paste0(pred_path, auprc_filename), header = TRUE, sep = '\t', row.names = 1, stringsAsFactors=FALSE)
}
# Add DRNApred stacked data
for (i in 1:length(ligands_list)) {
    ligand <- ligands_list[[i]]
    auprc_filename <- paste0(ligand, "_DRNApred_auprc.csv")
    auprc_comp_data <- read.csv(paste0(drnapred_path, auprc_filename), header = TRUE, sep = '\t', row.names = 1, stringsAsFactors=FALSE)
    auprc_data[[i]] <- cbind(auprc_data[[i]], auprc_comp_data)
    #names(auprc_data[[i]])[length(names(auprc_data[[i]]))]<-"ens-LIGAND"
}

In [40]:
for (i in 1:length(ligands_list)) {
    ligand <- ligands_list[[i]]
    ligand_data[[i]]$fold <- 1
}

In [41]:
save_to_file <- TRUE
models_data <- list()
pred_folds <- list()
labels_folds <- list()
pred_obj <- list()
pr_perf_obj <- list()
auprc_res <- list()
ac_obj <- list()
legend_str <- list()
positives_num <- list()
total_num <- list()
random_val <- list()
postivies_prob <- list()
negatives_prob <- list()
pr_plots_list <- list()

for (i in 1:length(ligands_list)) {

    #Split by the different models
    models_data[[i]] <- split(ligand_data[[i]], ligand_data[[i]]$model)
    models_names <- attributes(models_data[[i]])$name

    #init model lists
    pred_folds[[i]] <- list()
    labels_folds[[i]] <- list()
    pred_obj[[i]] <- list()
    pr_perf_obj[[i]] <- list()
    auprc_res[[i]] <- list()
    postivies_prob[[i]] <- list()
    negatives_prob[[i]] <- list()

    #For each model, split by different folds
    for (j in 1:length(models_names)) {

        model_name <- models_names[j]

        pred_folds[[i]][[j]] <- split(models_data[[i]][[model_name]]$prob, models_data[[i]][[model_name]]$fold)
        labels_folds[[i]][[j]] <- split(models_data[[i]][[model_name]]$obs, models_data[[i]][[model_name]]$fold)

        pred_obj[[i]][[j]] <- prediction(pred_folds[[i]][[j]], labels_folds[[i]][[j]])
        pr_perf_obj[[i]][[j]] <- performance(pred_obj[[i]][[j]],"prec","rec")

        #parse AUPRC
        div_num <- folds_num
        auprc_res[[i]][[j]] <- round(sum(auprc_data[[i]][[model_name]], na.rm=T)/div_num, 4)

        #PR curve
        if (j == 1) {
            if (save_to_file) {
                #pdf(paste0("plots/",ligands_list[[i]],"_pr.pdf"))
                #png(paste0("plots/",ligands_list[[i]],"_pr.png"))
                svg(paste0("comparison_plots/",ligands_list[[i]],"_pr.svg"))
            }
            #plot(perf_obj[[i]][[j]],col="grey82",lty=3)
            plot(pr_perf_obj[[i]][[j]],col=colors_list[j], lwd=2,avg="vertical", main=ligands_list[[i]], xlim=c(0, 1), ylim=c(0,1))
            #plot(roc_perf_obj[[i]][[j]],col=colors_list[j], lwd=3,avg="vertical", main="small-molecule")
        } else if (j == 6) {
            plot(pr_perf_obj[[i]][[j]],col=colors_list[j], lwd=3, lty=3, avg="vertical", add=TRUE)
        } else if (j == 7) {
            plot(pr_perf_obj[[i]][[j]],col=colors_list[j], lwd=3, lty=4, avg="vertical", add=TRUE)
        } else if (j == 8) {
            plot(pr_perf_obj[[i]][[j]],col=colors_list[j], lwd=3, lty=6, avg="vertical", add=TRUE)
        } else if (j == 9) {
            plot(pr_perf_obj[[i]][[j]],col=colors_list[j], lwd=3, lty=2, avg="vertical", add=TRUE)
        } else {
            
            plot(pr_perf_obj[[i]][[j]],col=colors_list[j], lwd=2,avg="vertical", xlim=c(0, 1), ylim=c(0,1), add=TRUE)
        }
    }
    #Compute random line
    postivies_prob[[i]][[j]] <- subset(models_data[[i]][[model_name]], obs==1)$prob
    negatives_prob[[i]][[j]] <- subset(models_data[[i]][[model_name]], obs==0)$prob
    positives_num[[i]] <- length(postivies_prob[[i]][[j]])
    total_num[[i]] <- length(negatives_prob[[i]][[j]]) + positives_num[[i]]
    random_val[[i]] <- positives_num[[i]]/total_num[[i]]
    #print(random_val[[i]])
    abline(h=random_val[[i]], col="grey38")

    #Add legend
    legend_str[[i]] <-  paste0(models_names,", AUC=", auprc_res[[i]])
    legend("topright", title= "Model", legend_str[[i]], lwd=c(2,2), lty=c(1,1), col=colors_list)
    grid(nx = NULL, ny = NULL, col = "lightgray", lty = "dotted")
    if (save_to_file) {
        dev.off()
    }
}
